In [2]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Seoul/data

/content/drive/MyDrive/Seoul/data


### 라이브러리 로드

In [4]:
import os
import json
import folium
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd #지도

import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup

### 데이터 로드

In [5]:
# 지도 데이터 로드

# 지도 시각화 geo.json 파일로드 및 서울시만 추출
with open('geo_sigungu.json', 'r') as f:
  data_gu = json.load(f)

In [12]:
feat,s_code = [],{}
for d in data_gu['features']:
  # print((d['properties']['code']))
  # break
  c, n = d['properties']['code'], d['properties']['name']
  if c < '20000':
    feat.append(d)
    s_code[n] = c

In [13]:
data_gu['features'] = feat
print(len(feat))

25


In [8]:
# 지도시각화 함수 - 컬럼, 지도데이터, 색상
def plot_gu(data, df, colname, color):
  m1 = folium.Map(location=[37.5502, 126.982], tiles='cartodbpositron', zoom_start=11)

  m1.choropleth(
      geo_data=data,
      name = colname,
      data = df,
      columns = ['시군구명', colname],
      key_on='properties.name',
      fill_color=color,
      highlight=True,
      fill_opacity=0.7,
      line_opacity=0.5,
      legend_name=colname
  )

  style_function = lambda x: {'fillColor': '#ffffff',
                              'color':'#000000',
                              'fillOpacity': 0.1,
                              'weight': 0.1}

  highlight_function = lambda x: {'fillColor': '#000000',
                                  'color':'#000000',
                                  'fillOpacity': 0.50,
                                  'weight': 0.1}

  SuburbName = folium.features.GeoJson(
      data,
      style_function=style_function,
      control=False,
      highlight_function=highlight_function,
      tooltip=folium.features.GeoJsonTooltip(
          fields=['name', 'code', colname],
          aliases=['시군구명','행정구역코드', colname],
          style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
          sticky=True
          )
  )
  m1.add_child(SuburbName)
  m1.keep_in_front(SuburbName)
  folium.LayerControl().add_to(m1)

  return m1

In [ ]:
# 행정동코드
code = pd.read_csv('./code_info.csv', encoding='cp949')
print(code.shape)
code.head()

(425, 2)


,행정구역코드,행정동코드
0,1101072,11110515
1,1101053,11110530
2,1101054,11110540
3,1101055,11110550
4,1101056,11110560


In [ ]:
# 거주인구특성
pop = pd.read_csv('0.거주인구특성.csv', encoding='cp949')
print(pop.shape)
pop.head()

(425, 11)


,행정동코드,시군구명,행정동명,RSPOP_CNT,노령층인구,장애인인구,차상위인구,노령층비율,장애인비율,차상위비율,행정구역코드
0,11110515,종로구,청운효자동,1843182,447811.0,228.0,1,0.242955,0.000124,5.425400e-07,1101072
1,11110530,종로구,사직동,1317129,570080.0,432.0,74,0.432820,0.000328,5.618280e-05,1101053
2,11110540,종로구,삼청동,394764,489999.0,542.0,65,1.241245,0.001373,1.646553e-04,1101054
3,11110550,종로구,부암동,1427008,527750.0,587.0,182,0.369830,0.000411,1.275396e-04,1101055
4,11110560,종로구,평창동,2595131,541058.0,505.0,172,0.208490,0.000195,6.627796e-05,1101056


In [ ]:
# 복지시설
wel = pd.read_csv('./복지시설.csv', encoding='cp949')
print(wel.shape)
wel.head()

(3220, 13)


,시설명,시설코드,시설종류명(시설유형),시설종류상세명(시설종류),자치구(시)구분,시설장명,시군구코드,시군구명,시설주소,정원(수용인원),현인원,전화번호,우편번호
0,서울꽃동네신내노인요양원,A0001,(노인) 노인요양시설,노인의료복지시설,자치구,최은숙,1126000000,중랑구,서울특별시 중랑구 신내로 194,234.0,234.0,02-490-2609,02052
1,청운양로원,A0002,(노인) 양로시설,노인주거복지시설,자치구,이종명,1111000000,종로구,서울특별시 종로구 비봉길 76 (구기동),57.0,50.0,02-379-9232,03001
2,청운노인요양원,A0003,(노인) 노인요양시설,노인의료복지시설,자치구,이종후,1111000000,종로구,서울특별시 종로구 비봉길 76 (구기동),45.0,45.0,02-3217-0057,03001
3,홍파양로원,A0004,(노인) 양로시설,노인주거복지시설,자치구,김우리,1135000000,노원구,서울특별시 노원구 동일로248길 30 (상계동),44.0,32.0,02-939-0735,01623
4,천사노인요양원,A0007,(노인) 노인요양시설,노인의료복지시설,자치구,김샛별,1150000000,강서구,서울특별시 강서구 강서로45다길 30-22,161.0,151.0,02-2602-2443,07704


In [ ]:
wel_gu = wel.groupby(['시군구명'], as_index=False)[['현인원']].count()
wel_gu.rename(columns={'현인원':'복지시설'}, inplace=True)
wel_gu.head()

,시군구명,복지시설
0,강남구,107
1,강동구,129
2,강북구,115
3,강서구,184
4,관악구,133


In [ ]:
# 노후도
old = pd.read_csv('./노후비율_구단위.csv', encoding='cp949')
print(old.shape)
old.head()

(25, 5)


,구분.1,합계,30~35년미만,35년이상,노후비율
0,강남구,12162,1609,2130,30.74%
1,강동구,15398,3799,3767,49.14%
2,강북구,22006,4131,7986,55.06%
3,강서구,18704,3493,3847,39.24%
4,관악구,24867,5531,5991,46.33%


In [ ]:
# 노후도 데이터 전처리
old['노후비율'] = old['노후비율'].apply(lambda x: float(x.replace('%','')))
old = old[['구분.1', '합계', '노후비율']]
old.columns = ['시군구명', '건물수', '노후비율']
old.head()

,시군구명,건물수,노후비율
0,강남구,12162,30.74
1,강동구,15398,49.14
2,강북구,22006,55.06
3,강서구,18704,39.24
4,관악구,24867,46.33


In [ ]:
# 버스정류장
bus = pd.read_csv('./버스정류소.csv', encoding='cp949')
print(bus.shape)
bus.rename(columns={'X좌표':'경도', 'Y좌표':'위도'}, inplace=True)
bus.head()

(11290, 6)


,노드 ID,정류소번호,정류소명,경도,위도,정류소 타입
0,100000001,1001,종로2가사거리,126.987752,37.569808,중앙차로
1,100000002,1002,창경궁.서울대학교병원,126.996522,37.579433,중앙차로
2,100000003,1003,명륜3가.성대입구,126.998251,37.582581,중앙차로
3,100000004,1004,종로2가.삼일교,126.987613,37.568579,중앙차로
4,100000005,1005,혜화동로터리.여운형활동터,127.001744,37.586243,중앙차로


In [ ]:
import geopandas as gpd #지도
map_shp = gpd.read_file('./map_sigungu/SIG.shp', encoding='cp949')
map_shp['geometry'].crs

<Projected CRS: PROJCS["PCS_ITRF2000_TM",GEOGCS["ITRF2000",DATUM[" ...>
Name: PCS_ITRF2000_TM
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Transverse Mercator
Datum: International Terrestrial Reference Frame 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [ ]:
map_shp.head()

,SIG_CD,SIG_ENG_NM,SIG_KOR_NM,geometry
0,11110,Jongno-gu,종로구,"POLYGON ((956615.453 1953567.199, 956621.579 1..."
1,11140,Jung-gu,중구,"POLYGON ((957890.386 1952616.746, 957909.908 1..."
2,11170,Yongsan-gu,용산구,"POLYGON ((953115.761 1950834.084, 953114.206 1..."
3,11200,Seongdong-gu,성동구,"POLYGON ((959681.109 1952649.605, 959842.412 1..."
4,11215,Gwangjin-gu,광진구,"POLYGON ((964825.058 1952633.250, 964875.565 1..."


In [ ]:
# 서울만 추출
seoul_map = map_shp[map_shp['SIG_CD'] < '20000'].drop_duplicates()
print(len(seoul_map))
display(seoul_map.head())

25


,SIG_CD,SIG_ENG_NM,SIG_KOR_NM,geometry
0,11110,Jongno-gu,종로구,"POLYGON ((956615.453 1953567.199, 956621.579 1..."
1,11140,Jung-gu,중구,"POLYGON ((957890.386 1952616.746, 957909.908 1..."
2,11170,Yongsan-gu,용산구,"POLYGON ((953115.761 1950834.084, 953114.206 1..."
3,11200,Seongdong-gu,성동구,"POLYGON ((959681.109 1952649.605, 959842.412 1..."
4,11215,Gwangjin-gu,광진구,"POLYGON ((964825.058 1952633.250, 964875.565 1..."


In [ ]:
# 위경도 변환 후 중심좌표 추출
# seoul_map['geometry'] = seoul_map['geometry'].set_crs('EPSG:5179')

seoul_map['center'] = seoul_map['geometry'].geometry.centroid
seoul_map['geometry'] = seoul_map['geometry'].to_crs(epsg=4326)
seoul_map['center'] = seoul_map['center'].to_crs(epsg=4326)

seoul_map['경도'] = seoul_map['center'].map(lambda x: x.xy[0][0])
seoul_map['위도'] = seoul_map['center'].map(lambda x: x.xy[1][0])

seoul_map.head()

,SIG_CD,SIG_ENG_NM,SIG_KOR_NM,geometry,center,경도,위도
0,11110,Jongno-gu,종로구,"POLYGON ((127.00864 37.58047, 127.00871 37.580...",POINT (126.97732 37.59492),126.977321,37.594917
1,11140,Jung-gu,중구,"POLYGON ((127.02314 37.57196, 127.02336 37.571...",POINT (126.99597 37.56014),126.995968,37.560144
2,11170,Yongsan-gu,용산구,"POLYGON ((126.96918 37.55566, 126.96917 37.554...",POINT (126.97991 37.53138),126.979907,37.531385
3,11200,Seongdong-gu,성동구,"POLYGON ((127.04341 37.57234, 127.04524 37.571...",POINT (127.04106 37.55103),127.041059,37.551030
4,11215,Gwangjin-gu,광진구,"POLYGON ((127.10166 37.57240, 127.10224 37.572...",POINT (127.08574 37.54672),127.085744,37.546720


In [ ]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

sigungu = []
for i, row in bus.iterrows():
  lat, lng = row['위도'], row['경도']
  res_n = np.nan
  for j, row2 in seoul_map.iterrows():
    n, g = row2['SIG_KOR_NM'], row2['geometry']
    if g.contains(Point(lng, lat)):
      res_n = n
      break
    sigungu.append(res_n)

len(sigungu)
bus['시군구명'] = sigungu

11290

In [ ]:
bus_g = bus.groupby('시군구명', as_index=False)['정류소명'].count()
bus_g.columns = ['시군구명', '버스정류소']
bus_g.head()

,시군구명,버스정류소
0,강남구,561
1,강동구,403
2,강북구,410
3,강서구,622
4,관악구,490


In [ ]:
# 생활인구
flow = pd.read_csv('./생활인구.csv', encoding='cp949')
print(flow.shape)
flow.head()

(425, 5)


,행정동코드,시도명,시군구명,행정동명,총생활인구수
0,11110515,서울,종로구,청운효자동,18028.580860
1,11110530,서울,종로구,사직동,24180.214950
2,11110540,서울,종로구,삼청동,5226.917908
3,11110550,서울,종로구,부암동,14750.631610
4,11110560,서울,종로구,평창동,17974.404000


In [ ]:
flow_gu = flow.groupby('시군구명', as_index=False)['총생활인구수'].sum()
flow_gu.columns = ['시군구명', '생활인구']
flow_gu.head()

,시군구명,생활인구
0,강남구,804282.957820
1,강동구,550787.719553
2,강북구,286544.520070
3,강서구,538115.188650
4,관악구,465291.644810


In [ ]:
# 주거환경특성 데이터 병합
env = bus_g.merge(old, on='시군구명')
env = env.merge(wel_gu, on='시군구명')
env = env.merge(flow_gu, on='시군구명')
env.head()

,시군구명,버스정류소,건물수,노후비율,복지시설,생활인구
0,강남구,561,12162,30.74,107,804282.957820
1,강동구,403,15398,49.14,129,550787.719553
2,강북구,410,22006,55.06,115,286544.520070
3,강서구,622,18704,39.24,184,538115.188650
4,관악구,490,24867,46.33,133,465291.644810


In [ ]:
# env.columns = ['시군구명', '대중교통', '건물수', '노후비율', '복지시설', '생활인구']
# env.to_csv('./0.거주환경정보.csv', encoding='cp949', index=False)

In [11]:
env = pd.read_csv('./0.gu_cluster.csv', encoding='cp949')
env.drop('cluster', axis=1, inplace=True)
env.head()

,시군구명,단위면적당 교통 접근성,노후비율,1인당 복지시설 수,단위면적당 녹지면적,빈집 비율,희망의집 수리 비율
0,강남구,15.113924,0.281025,0.000196,0.050825,0.066512,0.000431
1,강동구,16.958113,0.167270,0.000279,0.024634,0.034944,0.002906
2,강북구,17.838983,0.289182,0.000390,0.005875,0.024699,0.007921
3,강서구,15.585042,0.098263,0.000323,0.027810,0.040459,0.001706
4,관악구,16.875211,0.208941,0.000266,0.008541,0.011920,0.005232


In [15]:
env['코드'] = [s_code[n] for n in env['시군구명']]
env['코드'].unique()

array(['11230', '11250', '11090', '11160', '11210', '11050', '11170',
       '11180', '11110', '11100', '11060', '11200', '11140', '11130',
       '11220', '11040', '11080', '11240', '11150', '11190', '11030',
       '11120', '11010', '11020', '11070'], dtype=object)

In [16]:
for i, f in enumerate(data_gu['features']):
  try:
    env_r = env[env['시군구명'] == f['properties']['name']]

    for c in env.columns[1:-1]:
      data_gu['features'][i]['properties'][c] = str(env_r[c].values[0])
  except:
    data_gu['features'][i]['properties'][c] = 0

In [ ]:
map_path = '../map/'

In [18]:
bus_m = plot_gu(data_gu, env, '단위면적당 교통 접근성', 'Blues')
bus_m

/usr/local/lib/python3.10/dist-packages/folium/folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


In [19]:
build_m = plot_gu(data_gu, env, '1인당 복지시설 수', 'Blues')
# build_m.save(map_path+'build_map.html')
build_m

/usr/local/lib/python3.10/dist-packages/folium/folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


In [21]:
old_m = plot_gu(data_gu, env, '노후비율', 'Reds')
# old_m.save(map_path+'old_map.html')
old_m

/usr/local/lib/python3.10/dist-packages/folium/folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


In [22]:
flow_m = plot_gu(data_gu, env, '빈집 비율', 'Reds')
# flow_m.save(map_path+'flow_map.html')
flow_m

/usr/local/lib/python3.10/dist-packages/folium/folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


In [17]:
env.columns

Index(['시군구명', '단위면적당 교통 접근성', '노후비율', '1인당 복지시설 수', '단위면적당 녹지면적', '빈집 비율',
       '희망의집 수리 비율', '코드'],
      dtype='object')

In [25]:
wel_m = plot_gu(data_gu, env, '단위면적당 녹지면적', 'Blues')
# wel_m.save(map_path+'wel_map.html')
wel_m

/usr/local/lib/python3.10/dist-packages/folium/folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


In [27]:
wel_m = plot_gu(data_gu, env, '희망의집 수리 비율', 'Reds')
# wel_m.save(map_path+'wel_map.html')
wel_m

/usr/local/lib/python3.10/dist-packages/folium/folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(
